# SpectroChemPy Overview

Welcome to SpectroChemPy! This tutorial will give you a quick overview of the main features
and capabilities of the library. We'll cover:

1. Loading and displaying spectroscopic data
2. Basic data manipulation and plotting
3. Processing techniques (smoothing, baseline correction)
4. Advanced analysis methods

For more detailed information, check out:
* [Gallery of Examples](examples/index.rst)
* [User's Guide](../userguide/introduction/introduction.rst)
* [API Reference](../reference/index.rst)

## Getting Started

First, let's import SpectroChemPy. By convention, we use the alias `scp`:

In [ ]:
import spectrochempy as scp

## Working with NDDataset Objects

The `NDDataset` is the core data structure in SpectroChemPy. It's designed specifically
for spectroscopic data and provides:

* Multi-dimensional data support
* Coordinates and units handling
* Built-in visualization
* Processing methods

Let's load some example FTIR (Fourier Transform Infrared) data:

In [ ]:
ds = scp.read("irdata/nh4y-activation.spg")

### Exploring Your Data

SpectroChemPy provides multiple ways to inspect your data. Let's look at:

1. Basic information summary
2. Detailed metadata

In [ ]:
# Quick overview
scp.set_loglevel("INFO")  # to see information
scp.info_(ds)

In [ ]:
# Detailed information
ds

### Data Visualization

SpectroChemPy's plotting capabilities are built on matplotlib but provide
spectroscopy-specific features:

In [ ]:
_ = ds.plot()

### Data Selection and Manipulation

You can easily select specific regions of your spectra using intuitive slicing.
Here we select wavenumbers between 4000 and 2000 cm⁻¹:

In [ ]:
region = ds[:, 4000.0:2000.0]
_ = region.plot()

### Mathematical Operations

NDDataset supports various mathematical operations. Here's an example of baseline correction:

In [ ]:
# Make y coordinate relative to the first point
region.y -= region.y[0]
region.y.title = "Dehydration time"

# Subtract the last spectrum from all spectra
region -= region[-1]

# Plot with colorbar to show intensity changes
_ = region.plot(colorbar=True)

## Data Processing Techniques

SpectroChemPy includes numerous processing methods. Here are some common examples:

### 1. Spectral Smoothing

Reduce noise while preserving spectral features:

In [ ]:
smoothed = region.smooth(size=51, window="hanning")
_ = smoothed.plot(colormap="magma")

### 2. Baseline Correction

Remove baseline artifacts using various algorithms:

In [ ]:
# Prepare data
region = ds[:, 4000.0:2000.0]
smoothed = region.smooth(size=51, window="hanning")

# Configure baseline correction
blc = scp.Baseline()
blc.ranges = [[2000.0, 2300.0], [3800.0, 3900.0]]  # Baseline regions
blc.multivariate = True
blc.model = "polynomial"
blc.order = "pchip"
blc.n_components = 5

# Apply correction
_ = blc.fit(smoothed)
_ = blc.corrected.plot()

## Advanced Analysis as for instance IRIS Processing

IRIS (Iterative Regularized Inverse Solver) is an advanced technique for analyzing
spectroscopic data. Here's an example with CO adsorption data:

In [ ]:
# Load and prepare CO adsorption data
ds = scp.read_omnic("irdata/CO@Mo_Al2O3.SPG")[:, 2250.0:1950.0]

# Define pressure coordinates
pressure = [
    0.00300,
    0.00400,
    0.00900,
    0.01400,
    0.02100,
    0.02600,
    0.03600,
    0.05100,
    0.09300,
    0.15000,
    0.20300,
    0.30000,
    0.40400,
    0.50300,
    0.60200,
    0.70200,
    0.80100,
    0.90500,
    1.00400,
]
ds.y = scp.Coord(pressure, title="Pressure", units="torr")

# Plot the dataset
_ = ds.plot(colormap="magma")

In [ ]:
# Perform IRIS analysis
iris = scp.IRIS(reg_par=[-10, 1, 12])
K = scp.IrisKernel(ds, "langmuir", q=[-8, -1, 50])
iris.fit(ds, K)
_ = iris.plotdistribution(-7, colormap="magma")